In [1]:
import keras
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import tensorflow as tf
import keras.backend as K

F:\Code_Tools\Anaconda\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
train_file = 'E:\\study\\NLP seminar\\task3\\snli_1.0\\snli_1.0_train.txt'
test_file = 'E:\\study\\NLP seminar\\task3\snli_1.0\\snli_1.0_test.txt'
dev_file = 'E:\\study\\NLP seminar\\task3\\snli_1.0\\snli_1.0_dev.txt'
train = pd.read_csv(train_file, sep = '\t')
test = pd.read_csv(test_file, sep = '\t')
dev = pd.read_csv(dev_file, sep = '\t')

In [6]:
from keras.layers import *
from keras.models import Model
from keras.layers import Activation, Dense
lsen_len = 14#left sentence平均值，数据集的readme中拿到
rsen_len = 8#right sentence平均值，数据集的readme中拿到
embedding_size = 300
embedding_file = '../../data/wordvector/fasttext/wiki-news-300d-1M.vec'
vocab_size = 63035
lstm_dim = 256
compare_dim = 128
compare_dropout = 0.5

In [7]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [10]:
##########数据统计和噪声处理#############
label_dict = {"neutral":0,"contradiction":1,"entailment":2}
vocab_set = set()
lhs = []
rhs = []
for k,v in enumerate(train.sentence1):
    if type(v) == float:
        print(k,v)
        invalid_idx.append(k)
    else:
        v = v.split()
        lhs.append(v)
        for j in v:
            vocab_set.add(j)
invalid_idx = []
for k,v in enumerate(train.sentence2):
    if type(v) == float:
        print(k,v)
        invalid_idx.append(k)
    else:
        v = v.split()
        rhs.append(v)
        for j in v:
            vocab_set.add(j)
for k,v in enumerate(train.gold_label):
    if v not in label_dict.keys():
        invalid_idx.append(k)

print("词数：",len(vocab_set))
lhs.sort()
rhs.sort()
print("length of lhs and rhs:",len(lhs[int(0.5*len(lhs))]),len(rhs[int(0.5*len(rhs))]))

91479 nan
91480 nan
91481 nan
311124 nan
311125 nan
311126 nan
词数： 63035
length of lhs and rhs: 11 12


In [11]:
#处理输入格式
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
x1train = []
x2train = []
ytrain = []
for k,v in enumerate(train.sentence1):
    if k not in invalid_idx:
        x1train.append(v)
for k,v in enumerate(train.sentence2):
    if k not in invalid_idx:
        x2train.append(v)
for k,v in enumerate(train.gold_label):
    if k not in invalid_idx:
        ytrain.append(v)
x1train = pd.Series(x1train)
x2train = pd.Series(x2train)
ytrain = pd.Series(ytrain)
x1test = test.sentence1
x2test = test.sentence2

In [12]:
tk = Tokenizer()
tk.fit_on_texts(x1train.values)
x1_train = tk.texts_to_sequences(x1train)
x2_train = tk.texts_to_sequences(x2train)
x1_test = tk.texts_to_sequences(x1test)
x2_test = tk.texts_to_sequences(x2test)
x1_train = pad_sequences(x1_train, lsen_len)
x2_train = pad_sequences(x2_train, rsen_len)
x1_test = pad_sequences(x1_test, lsen_len)
x2_test = pad_sequences(x2_test, rsen_len)

In [30]:
########验证集数据处理##########
dev_invalid_idx = []
x1dev = []
x2dev = []
ydev = []
for k,v in enumerate(dev.sentence1):
    if type(v) == float:
        print(k,v)
        dev_invalid_idx.append(k)
for k,v in enumerate(dev.sentence2):
    if type(v) == float:
        print(k,v)
        dev_invalid_idx.append(k)
for k,v in enumerate(dev.gold_label):
    if v not in label_dict.keys():
        #print(k,v)
        dev_invalid_idx.append(k)

for k,v in enumerate(dev.sentence1):
    if k not in dev_invalid_idx:
        x1dev.append(v)
for k,v in enumerate(dev.sentence2):
    if k not in dev_invalid_idx:
        x2dev.append(v)
for k,v in enumerate(dev.gold_label):
    if k not in dev_invalid_idx:
        ydev.append(v)
x1_dev = tk.texts_to_sequences(x1dev)
x2_dev = tk.texts_to_sequences(x2dev)
x1_dev = pad_sequences(x1_dev, lsen_len)
x2_dev = pad_sequences(x2_dev, rsen_len)
y_dev = []
count = 0
for i in ydev:
    y_dev.append(label_dict[i])

In [14]:
################################ESIM Model##############################################
def build_model(embedding_matrix=None):
    ############输入编码##############
    q1_inputs = Input(name='q1',shape=(lsen_len,))
    q2_inputs = Input(name='q2',shape=(rsen_len,))#这里两个输入的长度不一定要保持一致，因为我们attention的是词和词
    if embedding_matrix is None:
        embedding = Embedding(vocab_size, embedding_size)
    else:
        embedding = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], trainable=False)
    bn = BatchNormalization(axis=2)
    q1_embed = bn(embedding(q1_inputs))
    q2_embed = bn(embedding(q2_inputs))
    q1_encoded = Bidirectional(LSTM(lstm_dim//2, return_sequences=True))(q1_embed)
    q2_encoded = Bidirectional(LSTM(lstm_dim//2, return_sequences=True))(q2_embed)
    #print("q1,q2 shape after bilstm:",q1_encode.shape,q2_encode.shape)#输出格式为(x1,x2,x3),x1是批数，x2是句子长度，x3是bilstm隐层变量
    ############局部推理(attention)##############
    def multiply(tensor):
        input_1 = tensor[0]
        input_2 = tensor[1]
        #in1_aligned = K.zeros([input_1.shape[0], input_1.shape[2]], dtype=tf.float32)
        for i in range(lsen_len):
            attention_1 = Dot(axes=-1)([input_1[:,i,:],input_2])#(bs,8)
            w_att_1 = Lambda(lambda x: keras.activations.softmax(x, axis=1),)(attention_1)#(bs,8)
            vector = Dot(axes=1)([w_att_1,input_2])#(bsx8)和(bsx8x256)作点乘，得到(bsx256)
            att_value = K.expand_dims(vector,axis=1)#把维度变成bsx1x256
            if i == 0:
                in1_aligned = att_value
            else:
                in1_aligned = tf.concat([in1_aligned,att_value],1)
        #print("shape of my heart1:",attention_1.shape,w_att_1.shape,vector.shape,att_value.shape,in1_aligned.shape)

        #in2_aligned = K.zeros([input_1.shape[0], input_1.shape[2]], dtype=tf.float32)
        for i in range(rsen_len):
            attention_2 = Dot(axes=-1)([input_1,input_2[:,i,:]])#(bs,14)
            w_att_2 = Lambda(lambda x: keras.activations.softmax(x, axis=1),)(attention_2)#(bs,14)
            vector = Dot(axes=1)([w_att_2,input_1])#(bsx14)和(bsx14x256)作点乘，得到(bsx256)
            att_value = K.expand_dims(vector,axis=1)#把维度变成bsx1x256
            if i == 0:
                in2_aligned = att_value
            else:
                in2_aligned = tf.concat([in2_aligned,att_value],1)
        #print("shape of my heart2:",attention_2.shape,w_att_2.shape,vector.shape,att_value.shape,in2_aligned.shape)
        #print(type(in1_aligned),type(in2_aligned))

        return [in1_aligned,in2_aligned]#这里如果不是return一个list会报错
    
    [in1_aligned, in2_aligned]=Lambda(multiply)([q1_encoded,q2_encoded])
    ############推理合成##############
    #print("shape:",in1_aligned.shape,in2_aligned.shape)
    # Compare
    print(K.tf.multiply(q1_encoded, in1_aligned))
    q1_combined = concatenate(
        [q1_encoded, in1_aligned, subtract([q1_encoded, in1_aligned]), Lambda(lambda x:K.tf.multiply(x[0],x[1]))([q1_encoded, in1_aligned])])
    q2_combined = concatenate(
        [q2_encoded, in2_aligned, subtract([q2_encoded, in2_aligned]), Lambda(lambda x:K.tf.multiply(x[0],x[1]))([q2_encoded, in2_aligned])])
    q1_dense1 = Dense(compare_dim, activation='relu')(q1_combined)
    q2_dense1 = Dense(compare_dim, activation='relu')(q2_combined)
    q1_compare = Dropout(compare_dropout)(q1_dense1)
    q2_compare = Dropout(compare_dropout)(q2_dense1)
#     dense2 = Dense(compare_dim, activation='relu')(drop1)
#     drop2 = Dropout(compare_dropout)(dense2)
    #q1_compare = drop2(q1_combined)
    #q2_compare = drop2(q2_combined)

    # Aggregate
    q1_maxp = MaxPooling1D()(q1_compare)
    q1_avgp = AveragePooling1D()(q1_compare)
    q2_maxp = MaxPooling1D()(q2_compare)
    q2_avgp = AveragePooling1D()(q2_compare)
    res = Concatenate(1)([q1_maxp,q1_avgp,q2_maxp,q2_avgp])
    flat = Flatten()(res)
    dense3 = Dense(128, activation='relu')(flat)
    preds = Dense(3, activation='softmax')(dense3)
    
    model = Model(inputs=[q1_inputs,q2_inputs], outputs=[preds])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
model = build_model()
model.summary()

Tensor("Mul:0", shape=(?, 14, 256), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
q1 (InputLayer)                 (None, 14)           0                                            
__________________________________________________________________________________________________
q2 (InputLayer)                 (None, 8)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         multiple             18910500    q1[0][0]                         
                                                                 q2[0][0]                         
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor multiple             1200 

In [16]:
model.fit([x1_train,x2_train], y_train, batch_size=32, epochs=7, verbose=1)

Epoch 1/7
549361/549361 [==============================] - 7282s 13ms/step - loss: 0.7560 - acc: 0.6697
Epoch 2/7
549361/549361 [==============================] - 7073s 13ms/step - loss: 0.6354 - acc: 0.7361
Epoch 3/7
549361/549361 [==============================] - 7346s 13ms/step - loss: 0.5751 - acc: 0.7663
Epoch 4/7
549361/549361 [==============================] - 7571s 14ms/step - loss: 0.5287 - acc: 0.7890
Epoch 5/7
549361/549361 [==============================] - 7127s 13ms/step - loss: 0.4879 - acc: 0.8071
Epoch 6/7
549361/549361 [==============================] - 7498s 14ms/step - loss: 0.4495 - acc: 0.8244
Epoch 7/7
549361/549361 [==============================] - 8009s 15ms/step - loss: 0.4165 - acc: 0.8384


In [34]:
model.evaluate([x1_dev,x2_dev], y_dev, batch_size=32, verbose=1)#loss and accuracy

9842/9842 [==============================] - 8s 811us/step


[0.601235988493301, 0.7725055883192866]

In [ ]:
#最后结果比paper提出的差不少，猜测原因：epoch太少，没有用预先的word embedding(paper中是Glove)，以及推理合成层那里稍有不同。